In [4]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt

In [5]:
r = 102317272
ar = 0.5 * (r % 7)
br = 0.3 * (r % 5 + 1)

In [6]:
df = pd.read_csv("data.csv", encoding="cp1252")
x = df["no2"].dropna().values

/tmp/ipython-input-547897064.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data.csv", encoding="cp1252")


In [7]:
z = x + ar * np.sin(br * x)
z = z.reshape(-1, 1)

In [8]:
print("ar =", ar)
print("br =", br)

ar = 0.5
br = 0.8999999999999999


In [9]:
device = "cuda" if torch.cuda.is_available() else "cpu"

class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(1, 32),
            nn.ReLU(),
            nn.Linear(32, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )

    def forward(self, x):
        return self.net(x)


class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(1, 32),
            nn.ReLU(),
            nn.Linear(32, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.net(x)

In [10]:
G = Generator().to(device)
D = Discriminator().to(device)

loss_fn = nn.BCELoss()

opt_G = torch.optim.Adam(G.parameters(), lr=0.001)
opt_D = torch.optim.Adam(D.parameters(), lr=0.001)

real_data = torch.tensor(z, dtype=torch.float32).to(device)

epochs = 3000
batch_size = 256

for epoch in range(epochs):

    idx = np.random.randint(0, len(real_data), batch_size)
    real = real_data[idx]

    noise = torch.randn(batch_size, 1).to(device)
    fake = G(noise)

    #Train Discriminator
    d_real = D(real)
    d_fake = D(fake.detach())

    loss_D = loss_fn(d_real, torch.ones_like(d_real)) + \
             loss_fn(d_fake, torch.zeros_like(d_fake))

    opt_D.zero_grad()
    loss_D.backward()
    opt_D.step()

    #Train Generator
    noise = torch.randn(batch_size, 1).to(device)
    fake = G(noise)

    g_fake = D(fake)

    loss_G = loss_fn(g_fake, torch.ones_like(g_fake))

    opt_G.zero_grad()
    loss_G.backward()
    opt_G.step()

    if epoch % 500 == 0:
        print(epoch, loss_D.item(), loss_G.item())

0 1.8964300155639648 0.67436683177948
500 1.31021249294281 0.7010416984558105
1000 1.4121544361114502 0.6528542041778564
1500 1.3882343769073486 0.6395025849342346
2000 1.3852885961532593 0.7163298726081848
2500 1.391047477722168 0.6807750463485718


In [11]:
with torch.no_grad():
    noise = torch.randn(10000, 1).to(device)
    fake_samples = G(noise).cpu().numpy().flatten()

In [12]:
plt.figure()
plt.hist(z.flatten(), bins=60, density=True)
plt.title("Real Distribution (z)")
plt.savefig("real_histogram.png")
plt.close()

In [13]:
plt.figure()
plt.hist(fake_samples, bins=60, density=True)
plt.title("Generator Learned PDF")
plt.savefig("gan_pdf.png")
plt.close()

In [14]:
plt.figure()
plt.hist(z.flatten(), bins=60, density=True, alpha=0.5, label="Real")
plt.hist(fake_samples, bins=60, density=True, alpha=0.5, label="GAN")
plt.legend()
plt.title("GAN vs Real Distribution")
plt.savefig("gan_comparison.png")
plt.close()